In [ ]:
from langchain.memory import ConversationBufferWindowMemory  
  
memory = ConversationBufferWindowMemory(  
	return_messages=True,  
	k=5, # 대화 버퍼 윈도우의 크기. 즉 몇 개의 메세지를 저장할지.  
)  
  
def add_message(input, output):  
	memory.save_context({'input': input}, {'output': output})  
  
# 대화 6개 저장하기 ( 숫자 자료형 형태는 안됨 )
# add_message 메서드에는 문자열만 가능함 
for i in range(1, 7):  
	add_message(str(i), str(i))  
  
# 메모리를 봤을 때, 제일 처음 저장한 1은 사라져 있음.  
memory.load_memory_variables({})  


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}